![DataStax Academy](https://s3.amazonaws.com/datastaxtraining/vq8Jr36Gk48v/datastax-academy.svg "DataStax Academy")

# Exercise 19 - Tuning Partitioning: Partitioning Rules

## Background

In this exercise, you'll be exploring partitioning properties for RDDs created from different sources and transformations.

You'll be working with the `videos` table:

and a text file, `/root/data/video-ids.csv`, which contains only one column: the video IDs for different movies. Some of the video IDs are orphaned, i.e. not in the `killrvideo.videos` table.

***

## Directions

#### 1. Create an RDD from the text file. Hint: map() each row to a UUID via java.util.UUID.fromString(). What do you expect the partitioner and number of partitions to be? Why? Write Scala code to discover the actual values. Were they what you expected them to be? Why or why not?

In [5]:
val rdd1 = sc.textFile("file:///root/data/video-ids.csv")
    .map(row => java.util.UUID.fromString(row))
println(rdd1.partitioner)
println(rdd1.partitions.size)

None
2


#### 2. Create an RDD from the killrvideo.videos table select()ing only the videoids. What do you expect the partitioner and number of partitions to be? Why? Write Scala code to discover the actual values. Were they what you expected them to be? Why or why not?

In [8]:
val rdd2 = sc.cassandraTable[java.util.UUID]("killr_video", "videos")
    .select("video_id")
println(rdd2.partitioner)
println(rdd2.partitions.size)

None
1


#### 3. Find the orphaned video IDs by using subtract(). What do you expect the partitioner and number of partitions to be? Why? Write Scala code to discover the actual values. Were they what you expected them to be? Why or why not?

In [9]:
val rdd3 = rdd1.subtract(rdd2)

println(rdd3.partitioner)
println(rdd3.partitions.size)

None
2


#### 4. Turn your RDD into a RDD of keys via keyBy() then sort using sortByKey(). What do you expect the partitioner and number of partitions to be? Why? Write Scala code to discover the actual values. Were they what you expected them to be? Why or why not?

In [10]:
val rdd4 = rdd3.keyBy(row => row).sortByKey()

println(rdd4.partitioner)
println(rdd4.partitions.size)

Some(org.apache.spark.RangePartitioner@daac8b75)
2


#### 5. Display the resulting orphaned video IDs.

In [12]:
rdd4.collect.foreach(println)

(9056808b-ca65-1bfb-9957-3bea148dfdce,9056808b-ca65-1bfb-9957-3bea148dfdce)
(907df86e-2208-18a8-90aa-6d837c659f2f,907df86e-2208-18a8-90aa-6d837c659f2f)
(9646278f-14bd-11e5-88ea-8438355b7e3a,9646278f-14bd-11e5-88ea-8438355b7e3a)
(9db57288-e51c-1ff1-805d-c5f1e49c2c8b,9db57288-e51c-1ff1-805d-c5f1e49c2c8b)
(fe3c4045-6f37-1223-81be-250dc60cffc8,fe3c4045-6f37-1223-81be-250dc60cffc8)
(2645e79c-14bd-11e5-a456-8438355b7e3a,2645e79c-14bd-11e5-a456-8438355b7e3a)
(264601a3-14bd-11e5-8c2e-8438355b7e3a,264601a3-14bd-11e5-8c2e-8438355b7e3a)
(2646123a-14bd-11e5-b9db-8438355b7e3a,2646123a-14bd-11e5-b9db-8438355b7e3a)
(26461a70-14bd-11e5-ad08-8438355b7e3a,26461a70-14bd-11e5-ad08-8438355b7e3a)
(2e8ecb4f-e92b-139b-8183-4df0e2a817bb,2e8ecb4f-e92b-139b-8183-4df0e2a817bb)
